# Monte Carlo On policy
## Sources 
- Reinforcement Learning : An Introduction, by Richard Sutton and Andrew Barto

Toujours basé sur le Blackjack, nous allons rechercher une politique optimum.


In [1]:
import gymnasium as gym
import pandas as pd
from collections import defaultdict
import random

Création de l'environnement Blackjack

In [2]:
env = gym.make('Blackjack-v1')

Dictionnaire pour stocker les Q values (action-valeur)

In [3]:
Q = defaultdict(float)

Dictionnaire pour le total des retours G poour chaque couple s,a

In [4]:
total_return = defaultdict(float)

Dictionnaire pour le cumul du nombre de passage pour tout les s,a

In [5]:
N = defaultdict(int)

## Définissons une politique epsilon-greedy
Paramètres d'entrés   
    - Q fonction (s,a)   
    - L'état courant   
    - epsilon   
 
Paramètre de sortie    
    - a action choisie

In [8]:
def epsilon_greedy_policy(state,Q,epsilon):
  
    if random.uniform(0,1) < epsilon:
        return env.action_space.sample() # exploration
    else:
        max_q=-1
        max_a=0
        for a in range(env.action_space.n):
            if (state,a) in Q:
                if Q[(state,a)] > max_q:
                    max_q=Q[(state,a)]
                    max_a=a
        return(max_a)
        # return max(list(range(env.action_space.n)), key = lambda x: Q[(state,x)]) # exploitation

In [31]:
Z=defaultdict(float)
Z[ (1 , 0) ] = 10
Z[ (1, 1) ] = 12
assert epsilon_greedy_policy(1,Z,0) == 1

1

## Générateur d'épisode
Entrée :
- Q fonction
- espilon pour notre politique aléatoire

Sortie :
- notre épisode sous la forma s,a ,s'

In [35]:
num_timesteps = 100

In [36]:
def generate_episode(Q,epsilon):
    
    episode = []
    
    state = env.reset()[0]
    
    for t in range(num_timesteps):
        
        # Sélection d'une action en fonction de notre politique
        action = epsilon_greedy_policy(state,Q,espilon)
        
        # envoie de l'action à l'environnement pour retour (s_, r, done)
        next_state, reward, done, truncated, info = env.step(action)
        
        # stockage dans la liste du triplet (état, action, récompense)
        episode.append((state, action, reward))
        
        if done:
            break
            
        state = next_state

    return episode

***
# Calcul de la politique optimale



In [60]:
num_iterations = 500000
espilon = 0.5

Nous allons initialiser une politique random au démarrage.      
Selon le modèle GPI, Notre Q fonction va tendre vers les vrais valeurs et donc notre politique également.<br>
Pour simplifier et parce que l'intéret est limité dans le cas du blackjack, nous ferons abstraction de gamma.

In [61]:
random.seed(1)
for i in range(num_iterations):
    
    # on génére un épisode
    episode = generate_episode(Q,espilon)
    
    # on stocker les pairs s,a de l'épisode
    all_state_action_pairs = [(s, a) for (s,a,r) in episode]
    
    # on stocke les récompense
    rewards = [r for (s,a,r) in episode]

    # Pour chaque t de l'épisode 
    for t, (state, action, reward) in enumerate(episode):

        # First visit : on ne prend en compte que le premier passage s,a
        if not (state, action) in all_state_action_pairs[0:t]:
            
            # Calcul de G avec y = 1
            R = sum(rewards[t:])
            
            # Cumul G
            total_return[(state,action)] = total_return[(state,action)] + R
            
            # Comptage du nombre de passage
            N[(state, action)] += 1

            # Calcul de Q value (s,a) par la moyenne des G cumulés sur N
            Q[(state,action)] = total_return[(state, action)] / N[(state, action)]

---
Regardons les résultats pour une situation données et suivant l'action choisie.

In [62]:
Q[((18,10,0),0)]

-0.23262295081967213

In [64]:
Q[((21,10,0),0)]

0.8879683675390698

In [59]:
((19,1,1),1) in Q

True

Logique :)

# Stockage du résultat dans un Dataframe

In [65]:
df = pd.DataFrame(Q.items(),columns=['state_action pair','value'])

In [66]:
df.head(11)

,state_action pair,value
0,"((21, 10, 1), 1)",-0.169492
1,"((16, 10, 0), 0)",-0.564878
2,"((16, 4, 0), 0)",-0.215698
3,"((14, 9, 0), 0)",-0.549296
4,"((21, 10, 1), 0)",0.920828
5,"((17, 8, 1), 1)",-0.092081
6,"((17, 8, 0), 0)",-0.384435
7,"((12, 10, 0), 0)",-0.585380
8,"((13, 10, 0), 1)",-0.579223
9,"((15, 7, 1), 0)",-0.455621


In [52]:
Q[((21,11,0),1)]

0.0

Nous avons un Q optimal ou proche.   
Notre politique est basée sur une approche gloutonne de Q donc elle même optimale !

***
## Exemple
J'ai en main 19 points (sans as) et le croupier 5 points visibles.

In [15]:
df[ df['state_action pair'] == ((19,5,False),0) ]

,state_action pair,value
152,"((19, 5, 0), 0)",0.434579


In [16]:
df[ df['state_action pair'] == ((19,5,False),1) ]

,state_action pair,value
154,"((19, 5, 0), 1)",-0.84


## Ma politique va donc choisir un stand (0) car la valeur de fonction d'action-état est la plus haute !